In [114]:
import pandas as pd
import tqdm

In [243]:
df = pd.DataFrame()

for i in range(2001,2020):
    filename = f'atp_matches_{i}.csv'
    curr = pd.read_csv(filename)
    df = pd.concat([df, curr])

df.reset_index()

,index,best_of,draw_size,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_ace,l_bpFaced,l_bpSaved,...,winner_age,winner_entry,winner_hand,winner_ht,winner_id,winner_ioc,winner_name,winner_rank,winner_rank_points,winner_seed
0,0,3,NaN,32.0,26.0,13.0,10.0,8.0,8.0,5.0,...,19.850000,NaN,R,180.0,103720,AUS,Lleyton Hewitt,7.0,2625.0,1
1,1,3,NaN,35.0,21.0,16.0,10.0,1.0,4.0,0.0,...,21.250000,Q,R,175.0,103451,GER,Bjorn Phau,208.0,184.0,NaN
2,2,3,NaN,43.0,30.0,12.0,12.0,5.0,8.0,4.0,...,20.450000,Q,R,185.0,103598,BEL,Xavier Malisse,127.0,307.0,NaN
3,3,3,NaN,27.0,15.0,13.0,9.0,3.0,6.0,3.0,...,22.750000,NaN,R,188.0,103163,GER,Tommy Haas,23.0,1145.0,5
4,4,3,NaN,35.0,26.0,8.0,8.0,0.0,12.0,8.0,...,30.750000,NaN,R,185.0,101727,AUS,Jason Stoltenberg,66.0,606.0,NaN
5,5,3,NaN,26.0,14.0,11.0,9.0,1.0,4.0,0.0,...,24.410000,NaN,R,178.0,102834,USA,Paul Goldstein,81.0,517.0,NaN
6,6,3,NaN,30.0,18.0,10.0,9.0,0.0,6.0,1.0,...,22.370000,NaN,R,175.0,103252,ESP,Alberto Martin,79.0,518.0,NaN
7,7,3,NaN,50.0,39.0,24.0,16.0,11.0,16.0,11.0,...,22.050000,NaN,R,170.0,103294,BEL,Christophe Rochus,74.0,540.0,NaN
8,8,3,NaN,29.0,20.0,4.0,8.0,4.0,5.0,1.0,...,26.930000,NaN,R,183.0,102318,ROU,Andrei Pavel,27.0,1105.0,6
9,9,3,NaN,32.0,23.0,8.0,10.0,2.0,4.0,1.0,...,21.790000,NaN,R,193.0,103344,CRO,Ivan Ljubicic,91.0,484.0,NaN


In [173]:
df.head()
print(df.columns)

Index(['best_of', 'draw_size', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms',
       'l_ace', 'l_bpFaced', 'l_bpSaved', 'l_df', 'l_svpt', 'loser_age',
       'loser_entry', 'loser_hand', 'loser_ht', 'loser_id', 'loser_ioc',
       'loser_name', 'loser_rank', 'loser_rank_points', 'loser_seed',
       'match_num', 'minutes', 'round', 'score', 'surface', 'tourney_date',
       'tourney_id', 'tourney_level', 'tourney_name', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_ace', 'w_bpFaced', 'w_bpSaved', 'w_df',
       'w_svpt', 'winner_age', 'winner_entry', 'winner_hand', 'winner_ht',
       'winner_id', 'winner_ioc', 'winner_name', 'winner_rank',
       'winner_rank_points', 'winner_seed'],
      dtype='object')


In [158]:
print(df.shape)
print(df.columns)

(58192, 49)
Index(['best_of', 'draw_size', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms',
       'l_ace', 'l_bpFaced', 'l_bpSaved', 'l_df', 'l_svpt', 'loser_age',
       'loser_entry', 'loser_hand', 'loser_ht', 'loser_id', 'loser_ioc',
       'loser_name', 'loser_rank', 'loser_rank_points', 'loser_seed',
       'match_num', 'minutes', 'round', 'score', 'surface', 'tourney_date',
       'tourney_id', 'tourney_level', 'tourney_name', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_ace', 'w_bpFaced', 'w_bpSaved', 'w_df',
       'w_svpt', 'winner_age', 'winner_entry', 'winner_hand', 'winner_ht',
       'winner_id', 'winner_ioc', 'winner_name', 'winner_rank',
       'winner_rank_points', 'winner_seed'],
      dtype='object')


Preprocessing
Keep relevant features
Clean and categorize data


In [290]:
print(df.isna().sum())

# Several matches do not have rankings for players.  I will assume these players were amateurs given wildcards or 
# or other low ranked players and assign a rank of 1000, the lowest possible rank on the ATP Tour

best_of             0
l_1stIn             0
l_1stWon            0
l_2ndWon            0
l_SvGms             0
l_ace               0
l_bpFaced           0
l_bpSaved           0
l_df                0
l_svpt              0
loser_age           0
loser_entry         0
loser_hand          0
loser_ht            0
loser_id            0
loser_ioc           0
loser_name          0
loser_rank          0
match_num           0
round               0
score               0
surface             0
tourney_date        0
tourney_id          0
tourney_level       0
tourney_name        0
w_1stIn             0
w_1stWon            0
w_2ndWon            0
w_SvGms             0
w_ace               0
w_bpFaced           0
w_bpSaved           0
w_df                0
w_svpt              0
winner_age          0
winner_entry        0
winner_hand         0
winner_ht           0
winner_id           0
winner_ioc          0
winner_name         0
winner_rank         0
winner_1yr_wins     0
winner_1yr_loses    0
dtype: int

In [244]:
#handle missing values for player specific data
df['loser_age'].fillna((df['loser_age'].mean()), inplace=True)
df['loser_ht'].fillna((df['loser_ht'].mean()), inplace=True)
df['loser_hand'].fillna('R', inplace=True)
df['loser_entry'].fillna('REG', inplace=True)
df['loser_rank'].fillna(1000, inplace=True)

df['winner_age'].fillna((df['loser_age'].mean()), inplace=True)
df['winner_ht'].fillna((df['winner_ht'].mean()), inplace=True)
df['winner_hand'].fillna('R', inplace=True)
df['winner_entry'].fillna('REG', inplace=True)
df['winner_rank'].fillna(1000, inplace=True)

In [245]:
#remove retirement and walkover matches for more accurate data
df = df[df['score'] != 'W/O']
df = df[~df.score.str.contains("RET", na=False)]
#Remove Davis Cup matches 
df = df[df['tourney_level'] != 'D']

In [246]:
drop_cols = ['winner_rank_points', 'winner_seed', 'minutes', 'draw_size', 'loser_rank_points', 'loser_seed']
df = df.drop(drop_cols, axis = 1)
df = df.dropna()

In [251]:
import datetime
df['tourney_date'] = df['tourney_date'].astype(str)
df['tourney_date'] = pd.to_datetime(df['tourney_date'])

In [154]:
df.shape

(50395, 43)

In [155]:
df.describe()

,best_of,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_ace,l_bpFaced,l_bpSaved,l_df,l_svpt,...,w_SvGms,w_ace,w_bpFaced,w_bpSaved,w_df,w_svpt,winner_age,winner_ht,winner_id,winner_rank
count,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,...,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000,50395.000000
mean,3.369362,48.767775,32.594444,15.171584,12.405973,5.170751,8.653517,4.788729,3.384383,81.574938,...,12.613235,7.006092,5.052962,3.475801,2.657049,78.553309,26.481146,185.862473,104869.506876,58.672468
std,0.776085,18.789162,14.121300,7.163326,4.105031,4.899582,4.089673,3.244931,2.503730,28.443354,...,4.092058,5.571386,3.995110,3.056210,2.263673,28.416857,3.832609,6.748714,6370.736413,74.943730
min,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,15.820000,163.000000,100644.000000,1.000000
25%,3.000000,35.000000,22.000000,10.000000,9.000000,2.000000,6.000000,2.000000,2.000000,60.000000,...,10.000000,3.000000,2.000000,1.000000,1.000000,57.000000,23.640000,183.000000,103484.000000,16.000000
50%,3.000000,45.000000,30.000000,14.000000,11.000000,4.000000,8.000000,4.000000,3.000000,76.000000,...,11.000000,6.000000,4.000000,3.000000,2.000000,73.000000,26.350000,185.000000,104198.000000,41.000000
75%,3.000000,59.000000,40.000000,19.000000,15.000000,7.000000,11.000000,7.000000,5.000000,97.000000,...,15.000000,9.000000,7.000000,5.000000,4.000000,94.000000,29.120000,190.000000,104918.000000,76.500000
max,5.000000,328.000000,284.000000,101.000000,91.000000,103.000000,34.000000,25.000000,23.000000,489.000000,...,90.000000,113.000000,30.000000,24.000000,26.000000,491.000000,40.624230,208.000000,206173.000000,1890.000000


In [103]:
df[df['l_svpt'] != 489].describe()

,best_of,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_ace,l_bpFaced,l_bpSaved,l_df,l_svpt,...,w_SvGms,w_ace,w_bpFaced,w_bpSaved,w_df,w_svpt,winner_age,winner_ht,winner_id,winner_rank
count,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,...,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000,50394.000000
mean,3.369330,48.762234,32.589455,15.169881,12.404413,5.168810,8.653411,4.788586,3.384034,81.566853,...,12.611700,7.003989,5.053002,3.475830,2.656904,78.545124,26.481173,185.862074,104869.513315,58.673255
std,0.776059,18.748129,14.076960,7.153186,4.090114,4.880211,4.089644,3.244804,2.502525,28.385672,...,4.077552,5.551398,3.995139,3.056234,2.263459,28.357680,3.832642,6.748185,6370.799459,74.944266
min,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,15.820000,163.000000,100644.000000,1.000000
25%,3.000000,35.000000,22.000000,10.000000,9.000000,2.000000,6.000000,2.000000,2.000000,60.000000,...,10.000000,3.000000,2.000000,1.000000,1.000000,57.000000,23.640000,183.000000,103484.000000,16.000000
50%,3.000000,45.000000,30.000000,14.000000,11.000000,4.000000,8.000000,4.000000,3.000000,76.000000,...,11.000000,6.000000,4.000000,3.000000,2.000000,73.000000,26.350000,185.000000,104198.000000,41.000000
75%,3.000000,59.000000,40.000000,19.000000,15.000000,7.000000,11.000000,7.000000,5.000000,97.000000,...,15.000000,9.000000,7.000000,5.000000,4.000000,94.000000,29.120000,190.000000,104918.000000,76.750000
max,5.000000,218.000000,171.000000,63.000000,50.000000,67.000000,34.000000,25.000000,23.000000,291.000000,...,49.000000,75.000000,30.000000,24.000000,26.000000,278.000000,40.624230,208.000000,206173.000000,1890.000000


Adding Features
- Need to use previous player performance to predict matchups
1. Previous Career High Rank


In [224]:
#TODO fix print order for loser first case

def addHeadToHead(tdf):
    matchtracker = {}
    tdf['head_to_head'] = "asdf"
    
    for id, row in tqdm.notebook.tqdm(tdf.iterrows(), total=tdf.shape[0]):
        winner = row['winner_id']
        loser = row['loser_id']
        
        if (winner, loser) in matchtracker:
            h2h = str(matchtracker[(winner, loser)])
            tdf.loc[id, 'head_to_head'] = h2h
            matchtracker[(winner, loser)][0]+=1
        elif (loser, winner) in matchtracker:
            h2h = str(matchtracker[(loser, winner)])
            w = matchtracker[(winner, loser)][0]
            l = matchtracker[(winner, loser)][1]
            tdf.loc[id, 'head_to_head'] = h2h
            matchtracker[(loser, winner)][1]+=1
        else:
            matchtracker[(winner, loser)] = [1,0]
            tdf.loc[id, 'head_to_head'] = '[0, 0]'
        
addHeadToHead(df)
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [267]:
!pip3 install numba

    100% |████████████████████████████████| 2.0MB 602kB/s ta 0:00:01
    100% |████████████████████████████████| 15.9MB 94kB/s eta 0:00:011  47% |███████████████▎                | 7.6MB 2.9MB/s eta 0:00:03    49% |████████████████                | 7.9MB 4.1MB/s eta 0:00:02    69% |██████████████████████▎         | 11.1MB 3.4MB/s eta 0:00:02    73% |███████████████████████▊        | 11.8MB 3.0MB/s eta 0:00:02
You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [284]:
# Window stats
# look at average performance metrics for past year
from dateutil.relativedelta import relativedelta

testdf = df[(df['winner_name'] == 'Roger Federer') | (df['loser_name'] == 'Roger Federer') ]

def addWindowStats(tdf):
    tdf['winner_1yr_wins'] = 
    tdf['winner_1yr_wins'] = -1
    tdf['winner_1yr_loses'] = -1
    for id, row in tqdm.notebook.tqdm(tdf.iterrows(), total=tdf.shape[0]):
        curr_date = row['tourney_date']
        one_year_ago = row['tourney_date'] - relativedelta(years=1)
        #mask = (tdf['tourney_date'] > (row['tourney_date'] - relativedelta(years=1))) & (tdf['tourney_date'] <= row['tourney_date']) & (tdf['winner_name'] == row['winner_name'])
        tdf.loc[id, 'winner_1yr_wins'] = len(tdf[(tdf['tourney_date'] > one_year_ago) & (tdf.index < id) & (tdf['winner_id'] == row['winner_id'])])
        tdf.loc[id, 'winner_1yr_loses'] = len(tdf[(tdf['tourney_date'] > one_year_ago) & (tdf['tourney_date'] <= curr_date) & (tdf['loser_id'] == row['winner_id'])])

addWindowStats(testdf)
        

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [291]:
def getpastyearwins(x):
    ser = df.player_id[(df.winner_id == x) & (df.index < x.index) & (df.tourney_date > (x.tourney_date - relativedelta(years=1)))]
    return ser.count()

df['winner_1yr_wins'] = df.apply(getpastyearwins)

AttributeError: ("'DataFrame' object has no attribute 'player_id'", 'occurred at index best_of')

In [191]:
test_df = df[df['winner_name'] == 'Roger Federer']

def addCareerHighRank(tdf):
    tdf['winner_high_rank'] = 1001
    tdf['loser_high_rank'] = 1001
    
    col_loc = tdf.columns.get_loc("winner_high_rank")
    
    for id, row in tqdm.tqdm(tdf.iterrows(), total=tdf.shape[0]):
        curr_df = tdf.iloc[0:id]
        curr_id = row['winner_id']
        curr_df = curr_df[curr_df['winner_id'] == curr_id]
        max_winner_rank = curr_df['winner_rank'].max()
        tdf.iat[id, col_loc] = max_winner_rank
        
for id, row in tqdm.tqdm(tdf.iterrows(), total=tdf.shape[0]):

SyntaxError: unexpected EOF while parsing (<ipython-input-191-e6cac5ec4ae6>, line 16)

In [294]:
testdf.winner_name[testdf.winner_name == 'Roger Federer']

135     Roger Federer
145     Roger Federer
160     Roger Federer
221     Roger Federer
321     Roger Federer
333     Roger Federer
339     Roger Federer
342     Roger Federer
343     Roger Federer
478     Roger Federer
490     Roger Federer
496     Roger Federer
619     Roger Federer
630     Roger Federer
636     Roger Federer
639     Roger Federer
893     Roger Federer
924     Roger Federer
939     Roger Federer
1107    Roger Federer
1137    Roger Federer
1152    Roger Federer
1408    Roger Federer
1425    Roger Federer
1609    Roger Federer
1647    Roger Federer
1666    Roger Federer
1675    Roger Federer
1696    Roger Federer
1706    Roger Federer
            ...      
1122    Roger Federer
1252    Roger Federer
1292    Roger Federer
1312    Roger Federer
1322    Roger Federer
1327    Roger Federer
1386    Roger Federer
1387    Roger Federer
1389    Roger Federer
1393    Roger Federer
1401    Roger Federer
1565    Roger Federer
1597    Roger Federer
1613    Roger Federer
1621    Ro